### Parse the expected outputs from RTF files and store them in a csv

In [5]:
import polars as pl

In [6]:
from os import listdir, path

year_directories = "../data/raw/Podatki - rtvslo.si"

years = listdir(year_directories)

file_list = []

for year in years:
    year_path = year_directories + "/" + year

    if path.isdir(year_path) != True:
        continue

    year_months = listdir(year_path)

    year_stripped = year.replace("Promet ", "")

    for month in year_months:
        month_path = year_path + "/" + month

        month_files = listdir(month_path)
        
        month_stripped = month.replace(" " + year_stripped, "")

        for single_file in month_files:

            file_stripped = single_file.replace(".rtf", "")

            file_list.append({
                "year": year_stripped,
                "month": month_stripped,
                "file_name": file_stripped,
                "path": month_path + "/" + single_file,
            })

In [7]:
from tqdm import tqdm
from striprtf import striprtf

outputs = {
    "year": [],
    "month": [],
    "file_name": [],
    "content": []
}

for item in tqdm(file_list):
    with open(item['path'], 'r') as file:
        raw_content = file.read()
        text = striprtf.rtf_to_text(raw_content)

        outputs["year"].append(item['year'])
        outputs["month"].append(item['month'])
        outputs["file_name"].append(item['file_name'])
        outputs["content"].append(text)

100%|██████████| 28037/28037 [00:30<00:00, 905.66it/s]


In [8]:
dfOutputs = pl.DataFrame(outputs)

In [9]:
dfOutputs.head()

year,month,file_name,content
str,str,str,str
"""2022""","""April""","""TMP-1""","""Prometne informacije 30.…"
"""2022""","""April""","""TMP-10""","""Prometne informacije 30.…"
"""2022""","""April""","""TMP-100""","""Prometne informacije 27.…"
"""2022""","""April""","""TMP-101""","""Prometne informacije 27.…"
"""2022""","""April""","""TMP-102""","""Prometne informacije 26.…"


In [10]:
dfOutputs.describe()

statistic,year,month,file_name,content
str,str,str,str,str
"""count""","""28037""","""28037""","""28037""","""28037"""
"""null_count""","""0""","""0""","""0""","""0"""
"""mean""",null,null,null,null
"""std""",null,null,null,null
"""min""","""2022""","""April""","""Promet""",""" 8.30 Podatki o prometu…"
"""25%""",null,null,null,null
"""50%""",null,null,null,null
"""75%""",null,null,null,null
"""max""","""2024""","""September""","""TMP9-2024-99""","""ujna prometna informacija. …"


In [11]:
dfOutputs.write_csv("../data/outputs.csv")